In [1]:
# import the GIS class in gis module
from arcgis import GIS
from arcpy import *
from arcpy.sa import *
from arcpy.ia import *
import arcpy
import os
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

# Setting up working environmentb
env.workspace = "C:\\Users\\mariu\\Documents\\ArcGIS\\Projects\\NEW_PORTWAIKATO2\\NEW_PORTWAIKATO2.gdb"
env.overwriteOutput = True
env.parallelProcessingFactor = "90%"


In [2]:
def get_geotagging(exif):
    if not exif:
        raise ValueError("No EXIF metadata found")

    geotagging = {}
    for (tag, value) in exif.items():
        
        tag_name = TAGS.get(tag, tag)
        
        if tag_name == "GPSInfo":
            for (key, val) in GPSTAGS.items():
                if key in value:
                    geotagging[val] = value[key]
    return geotagging

def get_decimal_from_dms(dms, ref):
    degrees = dms[0]
    minutes = dms[1]
    seconds = dms[2]
    decimal = degrees + (minutes / 60.0) + (seconds / 3600.0)

    if ref in ['S', 'W']:
        decimal = -decimal

    return decimal

def get_coordinates(geotags):
    if geotags['GPSLatitude']:
        lat = get_decimal_from_dms(geotags['GPSLatitude'], geotags['GPSLatitudeRef'])
        lon = get_decimal_from_dms(geotags['GPSLongitude'], geotags['GPSLongitudeRef'])
        return lat, lon
    else:
        raise ValueError("No GPS coordinates found in the image.")

def get_gps_location(image_path):
    try:
        image = Image.open(image_path)
        exif_data = image._getexif()
        geotags = get_geotagging(exif_data)
        if geotags:
            return get_coordinates(geotags)
        else:
            raise ValueError("No GPS coordinates found in the image.")
    except Exception as e:
        #print(f"Error: {e}")
        return None

def process_images_in_folder(folder_path):
    
    image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        coordinates = get_gps_location(image_path)
        if coordinates:
            #print(f"Image: {image_file}, Latitude: {coordinates[0]}, Longitude: {coordinates[1]}")
            row_value  = (image_file, (coordinates[0],coordinates[1] ))
            rows.append(row_value)

In [3]:
def addPointtoFeature():
    
    arcpy.DeleteRows_management(table)
    
    for row in rows:
        y = row[1][0]
        x = row[1][1]
        filename = row[0]

        with arcpy.da.InsertCursor(table, ['FILENAME', 'X','Y'],) as cursor:
            cursor.insertRow([filename, x, y])
        
    print("Point added successfully.")

In [4]:
# Example usage
rows = []

# Define the name of the new table
tablename = 'field_photo_location.dbf'

# Define the field names and data types for the table
# Format: field_name, field_data_type
fields = [('FILENAME', 'TEXT'), ('X', 'FLOAT'), ('Y', 'FLOAT')]

# Create the table using arcpy.CreateTable_management
#arcpy.CreateTable_management('D:\\OUTPUT_NP2', tablename)

table = "D:\\OUTPUT_NP2\\" + tablename
# Add fields to the table using arcpy.AddField_management
for field_name, field_data_type in fields:
    arcpy.AddField_management(table, field_name, field_data_type)
    

folder_path = r'D:\Field_Photo'
process_images_in_folder(folder_path)

addPointtoFeature()

output_feature = 'field_photos'

arcpy.management.XYTableToPoint(table, output_feature ,'X', 'Y', None, arcpy.SpatialReference(4167) )


inputField = "FILENAME"
matchTable = output_feature
matchField = "FILENAME"
pathField = "FILENAME" 
picFolder = folder_path

arcpy.management.EnableAttachments(output_feature)
# use the match table with the Add Attachments tool
arcpy.AddAttachments_management(output_feature, inputField, matchTable, matchField, 
                                pathField, picFolder)

Point added successfully.


<Result 'C:\\Users\\mariu\\Documents\\ArcGIS\\Projects\\NEW_PORTWAIKATO2\\NEW_PORTWAIKATO2.gdb\\field_photos'>